## Práctica 3. Optimización de modelos de aprendizaje automático con algoritmos genéticos

Cuando ajustamos un modelo de machine learning, los hiperparámetros (como la profundidad máxima de un árbol, la tasa de aprendizaje, el número de estimadores, etc.) juegan un papel crucial en el rendimiento del modelo. 
Seleccionar buenos valores para estos hiperparámetros es importante para mejorar la precisión del modelo y evitar el sobreajuste.
En esta práctica vamos a elegir los parámetros que optimizan la precisión del modelo usando un algoritmo genético. 

Vamos a utilizar la librería Scikit-learn (o sklearn), una biblioteca de machine learning para Python que proporciona herramientas simples y eficientes para el análisis y modelado de datos. Es ampliamente utilizada en la comunidad de ciencia de datos y aprendizaje automático debido a su facilidad de uso y la robustez de sus algoritmos.

In [ ]:
imprt numpy as np
imoport pandas as pd
from sklearn.datasets import load_wine, fetch_california_housing
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import ParameterGrid, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import time
from tqdm import tqdm
import random

import warnings
import sys
import os
from itertools import product
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses


## Carga de datasets
Puedes elegir cualquiera de los datasets de prueba que vienen con la librería sklearn 
https://scikit-learn.org/1.5/datasets/toy_dataset.html
A continuación tienes algunos ejemplos.

In [ ]:
# Cargar dataset wine que viene como ejemplo en sklearn: 
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine
dataset = load_wine()
X = dataset.data
y = dataset.target
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Cargar dataset de precios de casas que viene como ejemplo en sklearn (se binariza el target): 
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing
dataset = fetch_california_housing()
dataset.target = np.where(dataset.target > np.median(dataset.target), 1, 0)
X = dataset.data
y = dataset.target
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Tambien puedes utilizar tu propio archivo de datos en formato csv. En este ejemplo vamos a usar el dataset  titanic.csv que te puedes descargar de Kaggle (también lo puedes descargar del campus virtual).
Después los dividimos en dos partes, conjunto de entrenamiento y conjunto de prueba (para evaluar el modelo entrenado)

In [ ]:
# Cargar el dataset de Titanic, que se puede descargar de Kaggle:
#https://www.kaggle.com/c/titanic/data
dataset = pd.read_csv('titanic.csv')
X = dataset[dataset.columns[:-1]]
y = dataset[dataset.columns[-1:]]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelos que vamos a optimizar 

Puedes elegir cualquier de los dos modelos siguientes para optimizar: modelo con XGBoost o modelo con perceptron multicapa. Aprenderás más de los modelos de machine learning en otras asignaturas. Para esta práctica los modelos funcionan como cajas negras. 

## Modelo con XGBoost

XGBoost es una biblioteca optimizada y eficiente de machine learning basada en el algoritmo de Gradient Boosting. Su nombre es un acrónimo de "Extreme Gradient Boosting". XGBoost es conocido por su alto rendimiento y velocidad.
XGBoost se basa en el principio de Gradient Boosting, donde se construyen modelos secuencialmente, y cada nuevo modelo trata de corregir los errores cometidos por el modelo anterior. Específicamente en cada iteración, se ajusta un modelo débil (usualmente un árbol de decisión) a los residuos (errores) de los modelos anteriores. El objetivo es minimizar una función de pérdida usando el gradiente descendente, lo que le permite mejorar iterativamente la precisión del modelo.

En esta práctica vamos a elegir los valores de los parámetros que optimizan la precisión del modelo usando un algoritmo genético. 

In [ ]:
hyperparameter_space = {
    "gamma": np.arange(0, 0.5, 0.1),
    "learning_rate": np.array([0.01, 0.1, 0.2, 0.3, 0.4]),
    "max_depth": np.arange(3, 10, 1), 
    "n_estimators": np.arange(50, 750, 50), 
    "subsample": np.arange(0.4, 1.0, 0.1),
    "min_child_weight" : np.arange(0, 10, 1),
    "subsample": np.arange(0.4, 1.0, 0.1),
    "colsample_bytree": np.arange(0.4, 1.0, 0.1),
    "scale_pos_weight": np.arange(0, 50, 10),
}

# ParameterGrid crea un iterable de todas las combinaciones posibles de hiperparámetros dados en forma de diccionario.
# ParameterGrid se usa a menudo dentro de GridSearchCV

search_space = list(ParameterGrid(hyperparameter_space))
print('Combinations: ', len(search_space))

# Creamos el modelo. Todavía no está entrenado
xgb_model = xgb.XGBClassifier( n_jobs=-1, random_state=42)
model = xgb_model

# Entrenar el modelo
model.fit(X_train, y_train)


Esto sería un ejemplo de cómo crear un modelo XGBClassifier con algunos hiperparámetros personalizados
¿Cómo sabemos cuáles son los mejores valores? 
Se prueban y se evaluan modelos con distintas configuraciones de hiperparámetros. Hay 4410000 combinaciones para elegir. 

    model = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        min_child_weight=1,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=0,
        reg_alpha=0.01,
        reg_lambda=1,
        scale_pos_weight=1,
        objective='binary:logistic',
        n_jobs=-1,
        random_state=42
    )


## Modelo con Perceptron Multicapa
Este modelo es considerablemente más lento que XGBoost porque no está paralelizado

In [ ]:
first_layer_neurons = np.arange(10, 100, 10)
second_layer_neurons = np.arange(10, 100, 10)
hidden_layer_sizes = list(product(first_layer_neurons, second_layer_neurons))

hyperparameter_space = {
    "hidden_layer_sizes": hidden_layer_sizes,
    "activation": ['logistic', 'tanh', 'relu'],
    "solver": ['sgd', 'adam', 'lbfgs'],
    "alpha": 10.0 ** -np.arange(1, 10),
    "learning_rate": ['constant', 'adaptive']
}


search_space = list(ParameterGrid(hyperparameter_space))
print('Combinations: ', len(search_space))

# MLP is more "delicate" than XGBoost, so we need to preprocess the data removing NaNs and scaling it
imp = SimpleImputer( strategy='mean')
X = imp.fit_transform(X)
scaler = StandardScaler()
X = scaler.fit_transform(X)

mlp_model = MLPClassifier(max_iter = 1000, random_state=42)
model = mlp_model

# Busqueda de los mejores hiperparametros utilizando los métodos de scikit-learn

La librería Scikit-learn ofrece sus propios métodos de optimización de hiperparámetros, por ejemplo, recorrer exhaustivamente todas las combinaciones posibles (GridSearchCV), o usar RandomizedSearchCV que explora un número aleatorio de combinaciones de hiperparámetros. RandomizedSearchCV selecciona aleatoriamente un número especificado de combinaciones y evalúa el rendimiento del modelo para cada una de ellas. Esto reduce significativamente el tiempo de búsqueda, haciendo posible encontrar buenos (aunque no necesariamente óptimos) conjuntos de hiperparámetros con mayor rapidez. 
Aun así puedes comprobar que tarda bastante. Horas (dependiendo del dataset y del modelo). GridSearch es todavía más ineficiente.  Según el tamaño del espacio de hiperparámetros estas opciones no son aplicables. Vamos a ver si podemos mejorarlo con un algoritmo genético.

In [ ]:
start_time = time.time()
search = RandomizedSearchCV(model, param_distributions=hyperparameter_space, random_state=42, n_iter=200, cv=5, verbose=1, n_jobs=-1, return_train_score=True)
search.fit(X_train, y_train)
print("Time: %s seconds " % (time.time() - start_time))
print(search.best_score_)
print(search.best_params_)

In [ ]:
start_time = time.time()
search = GridSearchCV(model, param_grid=hyperparameter_space, cv=5, verbose=2, n_jobs=-1, return_train_score=True)
search.fit(X_train, y_train)
print("Time: %s seconds " % (time.time() - start_time))
print(search.best_score_)
print(search.best_params_)

# Búsqueda exhaustiva manual de los mejores hiperparámetros
Otra opción es programar una busqueda exhaustiva manual o probar aleatoriamente un subconjunto de las opciones. Pero, igual que las opciones anteriores, no son opciones muy útiles. 
    1: Búsqueda greedy: tarda varios días en ejecutarse (con suerte)

    2: búsqueda random: tarda unas horas en ejecutarse dependiendo del dataset pero los resultados no son buenos en general.

# Algoritmo genético 

Se pide resolver el problema de optimizar los hiperparámetros de alguno de los modelos propuestos usando un algoritmo genético. 
Puedes implementar tus propias variaciones de las funciones de selección, cruce y mutación o utilizar las que hemos visto en la sesión anterior. 
Evalua y comenta los resultados obtenidos y comparalos con otros métodos de optimización de parámetros. 

Analiza los resultados en términos de calidad y tiempo. 